## Imports 

In [1]:
import os 
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

## Read the dataset and the radiogenomic data

In [2]:
pyradiomics_dataset_path = os.path.join('..', 'dataset', 'dataset.csv')
pyradiomics_dataset = pd.read_csv(pyradiomics_dataset_path)
pyradiomics_dataset.head()

,Mask,Image,original_shape_VoxelVolume,original_shape_MeshVolume,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_Sphericity,original_shape_Maximum3DDiameter,original_shape_Maximum2DDiameterSlice,original_shape_Maximum2DDiameterColumn,...,lbp-3D-k_gldm_GrayLevelNonUniformity,lbp-3D-k_gldm_GrayLevelVariance,lbp-3D-k_gldm_HighGrayLevelEmphasis,lbp-3D-k_gldm_LargeDependenceEmphasis,lbp-3D-k_gldm_LargeDependenceHighGrayLevelEmphasis,lbp-3D-k_gldm_LargeDependenceLowGrayLevelEmphasis,lbp-3D-k_gldm_LowGrayLevelEmphasis,lbp-3D-k_gldm_SmallDependenceEmphasis,lbp-3D-k_gldm_SmallDependenceHighGrayLevelEmphasis,lbp-3D-k_gldm_SmallDependenceLowGrayLevelEmphasis
0,R01-127_roi.nii,R01-127.nii,6646.222923,6620.933051,3821.238084,0.577145,0.446231,42.351097,41.211769,38.654876,...,17568.232131,0.107344,1.366904,435.038264,447.886326,431.826248,0.908274,0.017466,0.060178,0.006788
1,R01-064_roi.nii,R01-064.nii,3366.088867,3297.932943,1870.232366,0.567092,0.572906,52.086588,14.925760,51.939907,...,814.020852,0.130997,1.465095,273.110607,297.173164,267.094968,0.883726,0.019400,0.065205,0.007949
2,R01-118_roi.nii,R01-118.nii,154492.126465,153863.610586,29776.235984,0.193524,0.466342,148.169042,75.007242,146.583008,...,27212.449782,0.174788,1.677255,310.474449,365.343731,296.757129,0.830686,0.015801,0.051458,0.006886
3,R01-044_roi.nii,R01-044.nii,707.652683,690.997678,1016.191502,1.470615,0.371957,32.898135,30.636614,26.940836,...,1678.344142,0.061102,1.196130,357.702929,361.049686,356.866240,0.950968,0.020960,0.064607,0.010049
4,R01-024_roi.nii,R01-024.nii,4905.527134,4877.243809,2747.608401,0.563353,0.506188,47.004263,27.938821,30.427050,...,8310.601509,0.143750,1.522119,362.528292,419.715021,348.231610,0.869470,0.011873,0.038674,0.005173


In [3]:
radiogenomics_labels_path = os.path.join('..', 'dataset', 'radiogenomics_labels.csv')
radiogenomics_labels = pd.read_csv(radiogenomics_labels_path)
radiogenomics_labels.tail()

,Case ID,Patient affiliation,Age at Histological Diagnosis,Weight (lbs),Gender,Ethnicity,Smoking status,Pack Years,Quit Smoking Year,%GG,...,Recurrence,Recurrence Location,Date of Recurrence,Date of Last Known Alive,Survival Status,Date of Death,Time to Death (days),CT Date,Days between CT and surgery,PET Date
206,R01-159,Stanford,75,184,Male,Caucasian,Former,55,1994.0,Not Assessed,...,no,NaN,NaN,7/13/1995,Alive,NaN,NaN,11/24/1994,14,11/16/1994
207,R01-160,VA,61,231.5,Male,Caucasian,Former,12,1993.0,Not Assessed,...,no,NaN,NaN,7/3/1999,Alive,NaN,NaN,8/12/1993,72,9/22/1993
208,R01-161,Stanford,52,Not Collected,Female,Caucasian,Former,7,NaN,Not Assessed,...,no,NaN,NaN,4/2/1999,Alive,NaN,NaN,12/13/1995,8,9/26/1995
209,R01-162,Stanford,67,158,Male,Asian,Former,15,1966.0,Not Assessed,...,no,NaN,NaN,10/8/1997,Dead,10/8/1997,671.0,10/3/1995,65,11/14/1995
210,R01-163,VA,68,229,Male,Caucasian,Current,30,NaN,Not Assessed,...,yes,distant,2/15/1996,1/11/1997,Dead,1/11/1997,462.0,8/17/1995,51,7/12/1995


## Data Preprocessing

In [4]:
pyradiomics_dataset['Case ID'] = None

for i, image in enumerate(pyradiomics_dataset['Image']):
    pyradiomics_dataset.loc[i, 'Case ID'] = image.split('.')[0]

In [5]:
dataset = pd.merge(pyradiomics_dataset, radiogenomics_labels[['Case ID', 'Survival Status']], left_on='Case ID', right_on='Case ID', how='left')

In [6]:
dataset.drop(['Mask', 'Image', 'Case ID'], axis=1, inplace=True)
dataset.dropna(inplace=True)

In [7]:
dataset.head()

,original_shape_VoxelVolume,original_shape_MeshVolume,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_Sphericity,original_shape_Maximum3DDiameter,original_shape_Maximum2DDiameterSlice,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_MajorAxisLength,...,lbp-3D-k_gldm_GrayLevelVariance,lbp-3D-k_gldm_HighGrayLevelEmphasis,lbp-3D-k_gldm_LargeDependenceEmphasis,lbp-3D-k_gldm_LargeDependenceHighGrayLevelEmphasis,lbp-3D-k_gldm_LargeDependenceLowGrayLevelEmphasis,lbp-3D-k_gldm_LowGrayLevelEmphasis,lbp-3D-k_gldm_SmallDependenceEmphasis,lbp-3D-k_gldm_SmallDependenceHighGrayLevelEmphasis,lbp-3D-k_gldm_SmallDependenceLowGrayLevelEmphasis,Survival Status
0,6646.222923,6620.933051,3821.238084,0.577145,0.446231,42.351097,41.211769,38.654876,42.075213,28.771746,...,0.107344,1.366904,435.038264,447.886326,431.826248,0.908274,0.017466,0.060178,0.006788,Alive
1,3366.088867,3297.932943,1870.232366,0.567092,0.572906,52.086588,14.925760,51.939907,50.219246,49.321299,...,0.130997,1.465095,273.110607,297.173164,267.094968,0.883726,0.019400,0.065205,0.007949,Alive
2,154492.126465,153863.610586,29776.235984,0.193524,0.466342,148.169042,75.007242,146.583008,146.015102,144.590257,...,0.174788,1.677255,310.474449,365.343731,296.757129,0.830686,0.015801,0.051458,0.006886,Alive
3,707.652683,690.997678,1016.191502,1.470615,0.371957,32.898135,30.636614,26.940836,21.011030,23.222462,...,0.061102,1.196130,357.702929,361.049686,356.866240,0.950968,0.020960,0.064607,0.010049,Alive
4,4905.527134,4877.243809,2747.608401,0.563353,0.506188,47.004263,27.938821,30.427050,34.059646,35.852307,...,0.143750,1.522119,362.528292,419.715021,348.231610,0.869470,0.011873,0.038674,0.005173,Dead


In [8]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

In [9]:
y.replace({'Alive': 1, 'Dead': 0}, inplace=True)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train.astype('float64')))
X_test = pd.DataFrame(scaler.transform(X_test.astype('float64')))

X_train.columns = X.columns
X_test.columns = X.columns

## Model Selection

In [25]:
cross_valid_scores = {}

### Decision Tree Classifier

In [35]:
from sklearn.tree import DecisionTreeClassifier

parameters = {
    "max_depth": [3, 5, 7, 9, 11, 13],
}

model_desicion_tree = DecisionTreeClassifier(
    random_state=42,
    class_weight='balanced',
)

model_desicion_tree = GridSearchCV(
    model_desicion_tree, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_desicion_tree.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_desicion_tree.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + \
    f'{model_desicion_tree.best_score_:.3f}'
)
cross_valid_scores['desicion_tree'] = model_desicion_tree.best_score_
print('-----')

-----
Best parameters {'max_depth': 3}
Mean cross-validated accuracy score of the best_estimator: 0.596
-----


C:\Users\Manos\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### Random Forest Classifier

In [32]:
%%time
from sklearn.ensemble import RandomForestClassifier

parameters = {
    "n_estimators": [5, 10, 15, 20, 25], 
    "max_depth": [3, 5, 7, 9, 11, 13],
}

model_random_forest = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
)

model_random_forest = GridSearchCV(
    model_random_forest, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_random_forest.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_random_forest.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_random_forest.best_score_:.3f}'
)
cross_valid_scores['random_forest'] = model_random_forest.best_score_
print('-----')

-----
Best parameters {'max_depth': 7, 'n_estimators': 15}
Mean cross-validated accuracy score of the best_estimator: 0.617
-----
Wall time: 4.02 s


C:\Users\Manos\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### XGBoost

In [40]:
%%time

from xgboost import XGBClassifier

parameters = {
    'max_depth': [3, 5, 7, 9], 
    'n_estimators': [5, 10, 15, 20, 25, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1]
}

model_xgb = XGBClassifier(
    random_state=42,
)

model_xgb = GridSearchCV(
    model_xgb, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_xgb.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_xgb.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_xgb.best_score_:.3f}'
)
cross_valid_scores['xgboost'] = model_xgb.best_score_
print('-----')

C:\Users\Manos\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-----
Best parameters {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 50}
Mean cross-validated accuracy score of the best_estimator: 0.691
-----
Wall time: 1min 14s


### LightGBM

In [64]:
%%time

import lightgbm as lgbm

parameters = {
    'n_estimators': [5, 10, 15, 20, 25, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [7, 15, 31],
}

model_lgbm = lgbm.LGBMClassifier(
    random_state=42,
    class_weight='balanced',
)

model_lgbm = GridSearchCV(
    model_lgbm, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_lgbm.fit(
    X_train, 
    y_train, 
)

print('-----')
print(f'Best parameters {model_lgbm.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_lgbm.best_score_:.3f}'
)
cross_valid_scores['lightgbm'] = model_lgbm.best_score_
print('-----')

C:\Users\Manos\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-----
Best parameters {'learning_rate': 0.05, 'n_estimators': 5, 'num_leaves': 7}
Mean cross-validated accuracy score of the best_estimator: 0.734
-----
Wall time: 2min 53s


## CatBoost Classifier

In [60]:
%%time
import catboost as cb

parameters = {
    'iterations': [10],
#     'iterations': [5, 10, 15, 20, 25, 50, 100],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'depth': [3, 5, 7, 9, 11, 13],
}

model_catboost = cb.CatBoostClassifier(
    verbose=False,
)

model_catboost = GridSearchCV(
    model_catboost, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_catboost.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_catboost.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_catboost.best_score_:.3f}'
)
cross_valid_scores['catboost'] = model_catboost.best_score_
print('-----')

C:\Users\Manos\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-----
Best parameters {'iterations': 100}
Mean cross-validated accuracy score of the best_estimator: 0.585
-----
Wall time: 20.2 s


## Logistic Regression

In [56]:
%%time
from sklearn.linear_model import LogisticRegression 

parameters = {
    "C": [0.001, 0.01, 0.1, 1.],
    "penalty": ["l1", "l2"]
}

model_logistic_regression = LogisticRegression(
    random_state=42,
    class_weight="balanced",
    solver="liblinear",
)

model_logistic_regression = GridSearchCV(
    model_logistic_regression, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_logistic_regression.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_logistic_regression.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_logistic_regression.best_score_:.3f}'
)
cross_valid_scores['logistic_regression'] = model_logistic_regression.best_score_
print('-----')

-----
Best parameters {'C': 0.1, 'penalty': 'l2'}
Mean cross-validated accuracy score of the best_estimator: 0.585
-----
Wall time: 646 ms


C:\Users\Manos\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### GaussianNB Classifier

In [58]:
from sklearn.naive_bayes import GaussianNB

gnb_clf = GaussianNB()

In [59]:
scores = cross_val_score(gnb_clf, X_train, y_train, scoring="accuracy", cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.55 (+/- 0.24)


### SVM Classifier

In [66]:
%%time

from sklearn.svm import SVC

parameters = {
    "C": [0.001, 0.01, 0.1, 1.],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "gamma": ["scale", "auto"],
}

model_svc = SVC(
    random_state=42,
    class_weight="balanced",
)

model_svc = GridSearchCV(
    model_svc, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_svc.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_svc.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_svc.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_svc.best_score_
print('-----')

-----
Best parameters {'C': 1.0, 'gamma': 'scale', 'kernel': 'poly'}
Mean cross-validated accuracy score of the best_estimator: 0.638
-----
Wall time: 2.32 s


C:\Users\Manos\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### AdaBoost Classifier

In [26]:
%%time

from sklearn.ensemble import AdaBoostClassifier

parameters = {
    "n_estimators": [5, 10, 15, 20, 25, 50, 75, 100], 
    "learning_rate": [0.001, 0.01, 0.1, 1.],
}

model_adaboost = AdaBoostClassifier(
    random_state=42,
)

model_adaboost = GridSearchCV(
    model_adaboost, 
    parameters, 
    cv=5,
    scoring='accuracy',
)

model_adaboost.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_adaboost.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_adaboost.best_score_:.3f}'
)
cross_valid_scores['ada_boost'] = model_adaboost.best_score_
print('-----')

-----
Best parameters {'learning_rate': 0.001, 'n_estimators': 5}
Mean cross-validated accuracy score of the best_estimator: 0.756
-----
Wall time: 54.7 s


### KNeighbors Classifier

In [64]:
from sklearn.neighbors import KNeighborsClassifier

kn_clf = KNeighborsClassifier()

In [65]:
scores = cross_val_score(kn_clf, X_train, y_train, scoring="accuracy", cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.55 (+/- 0.41)


### Gaussian Process Classifier

In [66]:
from sklearn.gaussian_process import GaussianProcessClassifier

gaussian_clf = GaussianProcessClassifier()

In [67]:
scores = cross_val_score(gaussian_clf, X_train, y_train, scoring="accuracy", cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.59 (+/- 0.23)


### SelectKBest

In [68]:
from sklearn.feature_selection import SelectKBest, chi2

In [69]:
k_best_clf = SelectKBest(chi2, k=50)
X_new = pd.DataFrame(k_best_clf.fit_transform(X_train, y_train))
X_new.shape

(94, 50)

In [70]:
X_new.columns = X_train.columns[k_best_clf.get_support()]

In [71]:
X_test_new = X_test.loc[:, X_new.columns]

## Final Model 

In [67]:
pd.DataFrame(cross_valid_scores, index=['cross_valid_score']).T

,cross_valid_score
random_forest,0.617021
desicion_tree,0.595745
ada_boost,0.755319
xgboost,0.691489
lightgbm,0.734043
catboost,0.585106
logistic_regression,0.585106
svc,0.638298


In [72]:
final_clf = XGBClassifier()
final_clf.fit(X_new, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [73]:
y_pred = final_clf.predict(X_test_new)
print("Accuracy: %0.2f" % (accuracy_score(y_test, y_pred)))

Accuracy: 0.70





## Save the final model 

In [75]:
import pickle

# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
output_path = os.path.join(os.getcwd(), 'models', pkl_filename)

with open(output_path, 'wb') as file:
    pickle.dump(final_clf, file)

In [81]:
# Load from file
with open(output_path, 'rb') as file:
    pickle_model = pickle.load(file)
    
# Calculate the accuracy score and predict target values
score = pickle_model.score(X_test_new, y_test)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(X_test_new)

Test score: 70.21 %




# Automated Hyperparameter Tuning
#### https://www.kaggle.com/pavansanagapati/automated-hyperparameter-tuning

In [74]:
# !pip install deap update_checker tqdm stopit

In [75]:
# !pip install tpot

In [16]:
from tpot import TPOTClassifier

parameters = {
               'learning_rate':  [0.001, 0.01, 0.1, 1.],
               'n_estimators': [5, 10, 15, 20, 25, 50, 75, 100]
}
               
tpot_classifier = TPOTClassifier(generations= 4, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict=
                                 {'sklearn.ensemble.AdaBoostClassifier': parameters}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(X_train, y_train)


Generation 1 - Current best internal CV score: 0.6716485507246377

Generation 2 - Current best internal CV score: 0.6716485507246377

Generation 3 - Current best internal CV score: 0.6716485507246377

Generation 4 - Current best internal CV score: 0.6716485507246377

Best pipeline: AdaBoostClassifier(input_matrix, learning_rate=0.01, n_estimators=15)


TPOTClassifier(config_dict={'sklearn.ensemble.AdaBoostClassifier': {'learning_rate': [0.001,
                                                                                      0.01,
                                                                                      0.1,
                                                                                      1.0],
                                                                    'n_estimators': [5,
                                                                                     10,
                                                                                     15,
                                                                                     20,
                                                                                     25,
                                                                                     50,
                                                                                     75,
          

In [27]:
y_pred = model_adaboost.predict(X_test)
print("Accuracy: %0.2f" % (accuracy_score(y_test, y_pred)))

Accuracy: 0.57


In [18]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)

0.574468085106383
